In [1]:
import pandas as pd
import subprocess
from pathlib import Path
import re
from tqdm import tqdm
from subprocess import Popen, PIPE, STDOUT
import swifter

## Import RawSource

In [2]:
FuncDef_11_wExtFuncCalls = pd.read_hdf('/mnt/md0/user/scheuererra68323/JTT/JTT_wExtFuncCalls.h5')

In [3]:
print(FuncDef_11_wExtFuncCalls.shape)
FuncDef_11_wExtFuncCalls.head()

(350765, 5)


,path,line_start,line_stop,code_snippet,external_function_names
0,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,25,28,[return 1;],[]
1,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,30,33,[return 0;],[]
2,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,37,50,"[wchar_t * data;, if(staticReturnsTrue()),...",[]
3,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,57,76,"[wchar_t * data;, if(staticReturnsTrue()),...",[printLine]
4,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,79,93,"[wchar_t * data;, if(staticReturnsTrue()),...",[]


## Run clang -dump-tokens on code snippet

In [4]:
df = FuncDef_11_wExtFuncCalls.copy().sample(n=25000)

def run_dump_tokens(_code_snippet):
    _code_snippet = '\n'.join(_code_snippet)
    _code_snippet = _code_snippet.encode('utf8') # encode code string to bytes for piping into clang
    
    p = Popen([r'clang', '-cc1', '-x', 'c++', '-dump-tokens'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
    return p.communicate(input=_code_snippet)[1].decode('utf8')

   #return subprocess.run([r'clang', '-cc1', '-x', 'c++', '-dump-tokens'], 
    #                                   shell=True,
     #                                  capture_output=True,
      #                                 encoding='utf-8',
       #                                input=_code_snippet
        #                              ).stderr

tqdm.pandas(desc="Dump Tokens")
df['dump_tokens_output'] = df.code_snippet.progress_apply(run_dump_tokens)

df.head()

Dump Tokens: 100%|██████████| 25000/25000 [30:14<00:00, 13.78it/s]


,path,line_start,line_stop,code_snippet,external_function_names,dump_tokens_output
83452,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,22,39,"[unsigned int data;, data = 0;, if(glo...","[printUnsignedLine, globalReturnsTrue]",unsigned 'unsigned'\t [StartOfLine]\tLoc=<<std...
248990,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,199,203,"[goodG2B();, goodB2G();]",[],identifier 'goodG2B'\t [StartOfLine]\tLoc=<<st...
220067,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,56,67,"[int64_t * data;, /* Initialize data*/, ...",[CWE762_Mismatched_Memory_Management_Routines_...,identifier 'int64_t'\t [StartOfLine]\tLoc=<<st...
283612,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,52,57,"[printIntLine(data->intOne);, /* POTENTIAL...",[],identifier 'printIntLine'\t [StartOfLine]\tLoc...
142227,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,46,52,"[if(GLOBAL_CONST_FIVE==5), {, sign...",[signal],if 'if'\t [StartOfLine]\tLoc=<<stdin>:1:1>\nl_...


In [5]:
print(df.shape)
print(df.dtypes)

(25000, 6)
path                       object
line_start                 object
line_stop                  object
code_snippet               object
external_function_names    object
dump_tokens_output         object
dtype: object


In [6]:
#print(df.iloc[0].path)
#print()
#print( "".join(df.iloc[0].code_snippet))#[0:500])
#print()
#print(df.iloc[0].dump_tokens_output[0:500])

## Store the output

In [7]:
import fastparquet
fastparquet.write("/mnt/md0/user/scheuererra68323/JTT/JTT_DumpTokens_wExtFuncCalls.parquet", df, row_group_offsets=1000, compression="gzip")

/home/scheuererra68323/.local/lib/python3.7/site-packages/fastparquet/encoding.py:222: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy8 = numba.jitclass(spec8)(NumpyIO)
/home/scheuererra68323/.local/lib/python3.7/site-packages/fastparquet/encoding.py:224: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  Numpy32 = numba.jitclass(spec32)(NumpyIO)
